# PLS (Partial Least Square / Projection to Latent Structures)

## 1. Introduction to PLS

PLS is a method to project a high-dimensional feature space into a lower-dimensional space while preserving as much relevant information as possible.

Unlike Principal Component Analysis (PCA) which is also a famous dimensionality reduction method, PLS not only look at the features $X$ but also the label $y$. 

Thus, PLS tries to make a feature extraction that explains both the variance within the features $X$ (like PCA) **AND** that are correlated with the labels $y$. 

*Why use PLS ?*: "The goal of PLS regression is to predict $Y$ from $X$ and to describe their common structure. When $Y$ is a vector and $X$ is full rank, this goal could be accomplished using ordinary multiple regression. When the number of predictors is large compared to the number of observations, $X$ is likely to be singular and the regression approach is no longer feasible (i.e., because of multicollinearity). Several approaches have been developed to cope with this problem. Several approaches have been developed to cope with this problem. One approach is to eliminate some predictors (e.g., using step-wise methods) another one, called principal component regression, is to perform a Principal Component Analysis (PCA) of the $X$ matrix and then use the principal components of $X$ as regressors on $Y$. The orthogonality of the principal components eliminates the multicolinearity problem. But, the problem of choosing an optimum subset of predictors remains. A possible strategy is to keep only a few of the first components. But they are chosen to explain $X$ rather than $Y$, and so, nothing guarantees that the principal components, which “explain” X, are relevant for $Y$." [1]


## 2. Step-by-step implementation
### 2.0 Data importation

To test our code, we will import a dummy dataset from scikit-learn.

In [200]:
from sklearn import datasets

# Load the Diabetes dataset
diabetes = datasets.load_diabetes()

# Extract features (X) and target variable (y)
X = diabetes.data
y = diabetes.target

### 2.1 Data preparation

"The properties of pls regression can be analyzed from a sketch of the original algorithm. The first step is to create two matrices: $E=X$ and $F=Y$. These matrices are then column centered and normalized (i.e., transformed into Z-scores)." [1]

In [201]:
import numpy as np 

def z_score_standardize(M):
    """Standardize a dataset matrix computing means and stds on columns

    Args:
        M (np.array): 2D-matrix

    Returns:
        np.array: standardized 2D-matrix
    """
    means = np.mean(M, axis=0)
    stds = np.std(M, axis=0)

    return (M - means) / stds

E, F = z_score_standardize(X.copy()), z_score_standardize(y.copy())

### 2.2 Base algorithm

The goal of the algorithm is to find the best straight line (of direction $w$) on which the projected points ($t$, called "scores")from $X$ maximize the covariance with $y$. 

"Before starting the iteration process, the vector $\textbf{u}$ is initialized with random values." [1]

#### Step 1: Estimate $X$ weights
**$w$ (the weight vector)** is the variable the algorithm optimizes. The optimization problem is defined as:

$$
\max_w \left( \text{Cov}(Xw, y) \right)^2 \quad \text{subject to} \quad \|w\|_2 = 1
$$

This is equivalent to maximizing the dot product:

$$
\max_w \langle Xw, y \rangle = \max_w \left( w^T X^T y \right)
$$

We solve this using the **Lagrangian**:

$$
\mathcal{L}(w, \lambda) = w^T X^T y - \lambda(w^T w - 1)
$$

Computing the derivative with respect to $w$:

$$
\frac{\partial \mathcal{L}}{\partial w} = X^T y - 2\lambda w = 0
$$

Which implies:

$$
w = \frac{1}{2\lambda} X^T y \quad \implies \quad w \propto X^T y
$$

**However**, this analytical solution  $w \propto X^Ty$  is strictly sufficient only in the PLS1 case (where $y$  is a single column vector). In this specific scenario, the relationship is linear and direct, and convergence is immediate.

In the general PLS2 case, where $y$ is a matrix with multiple columns, he target is not a fixed vector $y$ but a latent vector $u$ $\in \mathbb{R}^{n \times 1}$, which represents a linear combination of the columns of $y$.

Since $u$ depends on the weights of $y$, which in turn depend on the scores of $X$, we face a circular dependency:

$w \propto X^Tu$ and $u \propto Fc(w)$

*Therefore, the algorithm cannot solve for $w$ in a single step. Instead, it employs an iterative procedure (NIPALS). By alternating projections between the $X$  space and the $y$  space, the algorithm implements the Power Method. This iterative process is mathematically guaranteed to converge to the eigenvector associated with the largest eigenvalue of the cross-covariance matrix  $X^TYY^TX$, thus identifying the direction of maximum shared variance.*

#### Step 2: Estimate $t$ ($X$ factor scores)

$$
t \propto Ew
$$

This step is simply the computation of the projected coordinates of the points of $X$ ($\mathbb{R}^p \to \mathbb{R}^1$)

#### Step 3: Estimate $c$ ($y$ weights)

$$
c \propto F^Tt
$$

This step corresponds to the computation of the covariance between $y$ and the scores $t$. It tells us how each column of $y$ is explained by the current score $t$ (i.e. by the latent representation made).

#### Step 4: Estimate $u$ ($y$ scores)

$$
u = Fc \iff \forall i, u_i = \sum_{j=1}^q F_{ij}c_j
$$

At this step, we compute every $u_i$ which represents the linear combination of the labels $y$, weighted by their current importance. 

In [202]:
def base_PLS(E, F):
    if F.ndim == 1:
        """In the case of a $y$ with a single column, it is not useful to use the algorithm
        as we have an analytic answer to the problem."""   
        F = F.reshape(-1, 1)
        w = np.transpose(E) @ F
        w = w/np.linalg.norm(w, ord=2) # normalization to gauarantee ||w|| = 1

        t = E @ w

        c = np.transpose(F) @ t
        c = c / np.linalg.norm(c) # normalization
    else: 
        """Else, we need to use the PLS2 algorithm to estimate a solution. 
        """

        u = np.random.rand(E.shape[0]) # Initialize u with random values
        u = u.reshape(-1, 1) # Reshape for coherence in matrixes shapes

        for k in range(10): # To be changed with a real stop condition
            # Step 1: estimate X weights
            w = np.transpose(E) @ u
            w = w/np.linalg.norm(w, ord=2) # normalization

            # Step 2: Estimate t, the factor scores
            t = E @ w

            # Step 3: Estimate c (y weights)
            c = np.transpose(F) @ t
            c = c / np.linalg.norm(c) # normalization

            # Step 4: Estimate $y$ scores
            u = F @ c
        
    return w, t, c

### 2.3 Full algorithm

The idea is now to adapt the code so that it can be used to extract multiple components, not just the main one. 

#### Step 1: Apply the base PLS algorithm seen previously

#### Step 2: Compute $p$ 

We can explain this step by considering the first step as an encoding step. $w$ tells us how to encode our data in a more compact way while keeping as much information as possible. $p$ is what we need to "decode" the encoded information. It's the "opposite". 

Rigorously, we want to find $p$ to reconstruct $E$*:

$E \approx tp^T$

We thus have this optimisation problem**:

$$
min_p ||E - tp^T||_F^2
$$

"What is the $p$ minimizing the reconstruction error ?" 

$\Rightarrow p^T = (t^Tt)^{-1}t^TE \Rightarrow p = \frac{E^Tt}{t^Tt}$

<small>
* We cannot really compute the pseudo-invert of $w$ and simply have $t = Ew \Rightarrow E = tw^{*}$, this would cause orthogonality problems. 

\*\* Frobenius norm allows an easy derivation. Minimizing the Frobenius norm is equivalent to performing a standard Least Squares Linear Regression for each variable (column) of $X$ simultaneously
</small>

#### Step 3: Deflation

This step corresponds to the substraction of the information explained by the component extracted:

$$
E_{new} = E_{old} - tp^T
$$

where $tp^T$, as explained in the Step 2, is the "simplified" image of $E$. 

In [203]:
def PLS(E, F, n_components=3):
    E = E.copy()
    F = F.copy()

    if F.ndim == 1:
        F = F.reshape(-1, 1)

    # Initializations
    WW = np.zeros((E.shape[1], n_components)) # W: X weights
    TT = np.zeros((E.shape[0], n_components)) # T: X scores
    PP = np.zeros((E.shape[1], n_components)) # P: X loadings
    QQ = np.zeros((F.shape[1], n_components)) # Q: y loadings

    for k in range(n_components):
        # Step 1: Previously coded base PLS algorithm
        w, t, c = base_PLS(E, F)
        
        # Store results
        WW[:, k], TT[:, k] = w.reshape(-1), t.reshape(-1)

        # Step 2: Compute p
        p = (E.T @ t) / (t.T @ t)
        PP[:, k] = p.reshape(-1)
        
        q = (F.T @ t) / (t.T @ t) 
        QQ[:, k] = q.reshape(-1)

        # Deflation
        E = E - t @ p.T 
        F = F - t @ c.T 

    # Compute Beta
    rotation_matrix = WW @ np.linalg.pinv(PP.T @ WW)
    beta = rotation_matrix @ QQ.T

    return {
        "beta": beta,               
        "rotation_matrix": rotation_matrix, # To extract features T of a new dataset X
        "training_scores": TT,      # features T from train set (for analysis)
        "loadings_P": PP,     
        "weights_W": WW             
    }

PLS(E, F)

{'beta': array([[-0.00671583],
        [-0.15736633],
        [ 0.32931638],
        [ 0.19719245],
        [-0.02852215],
        [-0.07935239],
        [-0.12684228],
        [ 0.07462938],
        [ 0.27045501],
        [ 0.0676613 ]]),
 'rotation_matrix': array([[-0.15555647, -0.15918207,  0.18952191],
        [-0.0356518 , -0.44194866,  0.60701945],
        [-0.4855326 ,  0.52526487, -0.19423333],
        [-0.36551068,  0.26588359,  0.04778442],
        [-0.17553722, -0.37878337, -0.33914666],
        [-0.14410209, -0.49482996, -0.24268864],
        [ 0.32685311, -0.1159892 , -0.25221622],
        [-0.35637967, -0.15492979,  0.00229313],
        [-0.46850436,  0.30070811, -0.37719118],
        [-0.31666494, -0.02393712,  0.47622418]]),
 'training_scores': array([[-1.04908728,  1.16869634,  0.95697385],
        [ 2.95072957, -0.46628332, -1.01320231],
        [-0.51371155,  0.54017185,  1.18816627],
        ...,
        [ 0.30475429, -0.45524956,  1.89102775],
        [-0.99442812,

## 3. Comparison to skikit-learn

To make sure that the previous implementation is right, we can make a simple linear regression on some data using both the scikit-learn library and our implementation. We will check by comparing how the linear regression works after extracting the features using PLS in both cases

In [204]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Configuration
n_components = 3

####################
# Scikit-learn:
####################

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test = z_score_standardize(X_train.copy()), z_score_standardize(X_test.copy())
y_train, y_test = z_score_standardize(y_train.copy()), z_score_standardize(y_test.copy())

# Initialize PLS model with the desired number of components
pls_model = PLSRegression(n_components=n_components)

# Fit the model on the training data
pls_model.fit(X_train, y_train)

T_train = pls_model.x_scores_ 

# Predictions on the test set
y_pred = pls_model.predict(X_test)
T_test = pls_model.transform(X_test) 

# Evaluate the model performance
print("#"*70)
print("Scikit-Learn Performance: ")
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print("Samples from T_train and T_test:")
print(T_train[:5])
print(T_test[:5])

####################
# Implementation from-scratch:
####################
pls_results = PLS(X_train, y_train, n_components=n_components)
y_pred_scratch = X_test @ pls_results["beta"]
mse_scratch = mean_squared_error(y_test, y_pred_scratch)

print("#"*70)
print("Implementation from-scratch Performance: ")
print(f"Mean Squared Error: {mse_scratch}")

######################################################################
Scikit-Learn Performance: 
Mean Squared Error: 0.5338921572218949
Samples from T_train and T_test:
[[ 1.78007629  0.8767078   0.24507304]
 [ 0.58117233  0.88484495  1.05141395]
 [ 1.22494241 -1.93462658 -0.80106168]
 [-2.96148717 -0.85667361 -1.33923066]
 [-3.35428272 -0.03443725 -0.31127994]]
[[ 1.38452991  2.16582274 -1.9756764 ]
 [ 0.28502179 -1.34667062 -0.30881613]
 [ 1.12752396  1.66927976 -0.84066678]
 [ 5.02256128 -0.76748104 -0.57587736]
 [ 0.52840036  1.57810109  0.24706615]]
######################################################################
Implementation from-scratch Performance: 
Mean Squared Error: 0.5338921572218946


Results are very close, which means that most probably, our implementation works fine. 

## References

[1] "Partial Least Squares (PLS) Regression" Hervé Abdi